In [43]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [44]:
def load_file_with_dynamic_header(file_path):
    # Read raw file without headers
    raw_df = pd.read_excel(file_path, header=None)

    header_keywords = [
        "actividad/proyecto",
        "pia",
        "pim"
    ]

    matches = raw_df.apply(
        lambda row: any(
            row.astype(str)
               .str.lower()
               .str.contains(k, regex=False)
               .any()
            for k in header_keywords
        ),
        axis=1
    )

    if not matches.any():
        raise ValueError(f"No header row containing 'Funcion' found in {file_path}")

    header_row_idx = matches.idxmax()

    # Reload file using detected header
    return pd.read_excel(file_path, skiprows=header_row_idx)


def load_spending_data_for_year(base_dir, year):
    base_dir = Path(base_dir)
    year_folder = base_dir / f"SMPCT_{year}"

    if not year_folder.exists():
        raise FileNotFoundError(f"Folder not found: {year_folder}")

    dfs = []

    for file in year_folder.glob(f"*.xlsx"): # for file in year_folder.glob("*.xlsx"):
        filename = file.stem  # filename without .xlsx
        parts = filename.split("_")

        # Function extraction logic
        # Examples:
        # MPCT_FUNCION_2009 → general
        # MPCT_FUNCION_2009_SALUD → salud
        if len(parts) <= 3:
            function = "general"
        else:
            function = "_".join(parts[3:]).lower()

        try:
            df = load_file_with_dynamic_header(file)

            # Traceability fields
            df["year"] = int(year)
            df["function"] = function
            df["source_file"] = file.name

            dfs.append(df)

        except Exception as e:
            print(f"Error reading {file.name}: {e}")

    if not dfs:
        return pd.DataFrame()

    return pd.concat(dfs, ignore_index=True)


In [45]:
df_2007 = load_spending_data_for_year(r"D:\cg_inter_stuff\thesis_project\spending_data", 2007)


df_2007.head()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12848\1193674201.py:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(dfs, ignore_index=True)


,Producto / Proyecto,PIA,PIM,Certificación,Compromiso Anual,Ejecución,Unnamed: 6,Unnamed: 7,Avance %,year,function,source_file,Funcion,Unnamed: 4,Unnamed: 5,Actividad/Proyecto
0,NaN,NaN,NaN,NaN,NaN,Compromiso,NaN,NaN,NaN,2007,general,MPCT_FUNCION_2007.xlsx,NaN,Devengado,Girado,NaN
1,NaN,1819517.0,2449205.0,NaN,NaN,2161468,NaN,NaN,88.3,2007,general,MPCT_FUNCION_2007.xlsx,"03: PLANEAMIENTO, GESTIÓN Y RESERVA DE CONTING...",2161468,2103220,NaN
2,NaN,140000.0,128134.0,NaN,NaN,128131,NaN,NaN,100.0,2007,general,MPCT_FUNCION_2007.xlsx,10: AGROPECUARIA,128131,124431,NaN
3,NaN,594781.0,1064751.0,NaN,NaN,979015,NaN,NaN,91.9,2007,general,MPCT_FUNCION_2007.xlsx,23: PROTECCIÓN SOCIAL,979015,977247,NaN
4,NaN,1082528.0,664767.0,NaN,NaN,609944,NaN,NaN,91.8,2007,general,MPCT_FUNCION_2007.xlsx,22: EDUCACIÓN,609944,554999,NaN


In [46]:
df_2007 = df_2007[['year', 'Actividad/Proyecto', 'function', 'Funcion', 'source_file', 'PIA', 'PIM', 'Ejecución', 'Unnamed: 4', 'Unnamed: 5', 'Avance %']]
df_2007.head()

,year,Actividad/Proyecto,function,Funcion,source_file,PIA,PIM,Ejecución,Unnamed: 4,Unnamed: 5,Avance %
0,2007,NaN,general,NaN,MPCT_FUNCION_2007.xlsx,NaN,NaN,Compromiso,Devengado,Girado,NaN
1,2007,NaN,general,"03: PLANEAMIENTO, GESTIÓN Y RESERVA DE CONTING...",MPCT_FUNCION_2007.xlsx,1819517.0,2449205.0,2161468,2161468,2103220,88.3
2,2007,NaN,general,10: AGROPECUARIA,MPCT_FUNCION_2007.xlsx,140000.0,128134.0,128131,128131,124431,100.0
3,2007,NaN,general,23: PROTECCIÓN SOCIAL,MPCT_FUNCION_2007.xlsx,594781.0,1064751.0,979015,979015,977247,91.9
4,2007,NaN,general,22: EDUCACIÓN,MPCT_FUNCION_2007.xlsx,1082528.0,664767.0,609944,609944,554999,91.8


In [47]:
df_2015 = load_spending_data_for_year(r"D:\cg_inter_stuff\thesis_project\spending_data", 2015)


df_2015.head()

Error reading ~$MPCT_FUNCION_2015_viviendaDesarrolloUrbano.xlsx: [Errno 13] Permission denied: 'D:\\cg_inter_stuff\\thesis_project\\spending_data\\SMPCT_2015\\~$MPCT_FUNCION_2015_viviendaDesarrolloUrbano.xlsx'


,Funcion,PIA,PIM,Certificación,Compromiso Anual,Ejecución,Unnamed: 6,Unnamed: 7,Avance %,year,function,source_file,Producto / Proyecto,Actividad / Acción de Inversión / Obra
0,NaN,NaN,NaN,NaN,NaN,Atención de Compromiso Mensual,Devengado,Girado,NaN,2015,general,MPCT_FUNCION_2015.xlsx,NaN,NaN
1,"03: PLANEAMIENTO, GESTION Y RESERVA DE CONTING...",4045504.0,5862099.0,5025437.0,4992124.0,4991071,4987126,4960832,85.1,2015,general,MPCT_FUNCION_2015.xlsx,NaN,NaN
2,05: ORDEN PUBLICO Y SEGURIDAD,317860.0,115146.0,101283.0,101283.0,101283,101283,101283,88.0,2015,general,MPCT_FUNCION_2015.xlsx,NaN,NaN
3,10: AGROPECUARIA,0.0,2045223.0,1877232.0,1875578.0,1875578,1875178,1875178,91.7,2015,general,MPCT_FUNCION_2015.xlsx,NaN,NaN
4,12: ENERGIA,0.0,61369.0,34722.0,34722.0,34722,34722,34722,56.6,2015,general,MPCT_FUNCION_2015.xlsx,NaN,NaN


In [48]:
#df_2015 = df_2015[['year', 'Producto / Proyecto', 'Actividad / Acción de Inversión / Obra', 'function', 'Funcion', 'source_file', 'PIA', 'PIM', 'Ejecución', 'Unnamed: 6', 'Unnamed: 7', 'Avance %']]
#df_2015

In [49]:
df_2024 = load_spending_data_for_year(r"D:\cg_inter_stuff\thesis_project\spending_data", 2024)


df_2024.head()

,Funcion,PIA,PIM,Certificación,Compromiso Anual,Ejecución,Unnamed: 6,Unnamed: 7,Avance %,year,function,source_file,Producto / Proyecto,Actividad / Acción de Inversión / Obra
0,NaN,NaN,NaN,NaN,NaN,Atención de Compromiso Mensual,Devengado,Girado,NaN,2024,general,MPCT_FUNCION_2024.xlsx,NaN,NaN
1,"03: PLANEAMIENTO, GESTION Y RESERVA DE CONTING...",15458879.0,13876625.0,13743625.0,13584571.0,13578391,13535937,13513748,97.5,2024,general,MPCT_FUNCION_2024.xlsx,NaN,NaN
2,05: ORDEN PUBLICO Y SEGURIDAD,406092.0,528536.0,520263.0,519673.0,519673,519673,519673,98.3,2024,general,MPCT_FUNCION_2024.xlsx,NaN,NaN
3,07: TRABAJO,400000.0,1116240.0,1096783.0,1087356.0,1087356,1051121,1050841,94.2,2024,general,MPCT_FUNCION_2024.xlsx,NaN,NaN
4,08: COMERCIO,0.0,24500.0,24500.0,24500.0,22117,22117,22117,90.3,2024,general,MPCT_FUNCION_2024.xlsx,NaN,NaN


In [50]:
import pandas as pd
import re

def process_siaf_dataframe(df):
    """
    Takes a SIAF dataframe, identifies the column containing project descriptions,
    and extracts structured columns for a dimensional model.
    """
    
    # 1. Identify which column has the project text
    # We look for common names found in your raw tables
    target_cols = ['Producto / Proyecto', 'Funcion', 'Actividad/Proyecto', 'Actividad / Acción de Inversión / Obra']
    source_col = next((col for col in target_cols if col in df.columns), None)
    
    if not source_col:
        print("Warning: No project description column found.")
        return df

    def extract_details(text):
        if pd.isna(text) or text == "" or not any(char.isdigit() for char in str(text)[:7]):
            return pd.Series([None] * 7)
        
        text = str(text).strip()
        
        # A. Extract CUI (7 digits at the start)
        cui_match = re.search(r'^(\d{7})', text)
        cui = cui_match.group(1) if cui_match else None
        
        # Remove CUI and colon to clean the name
        clean_name = re.sub(r'^\d{7}\s*[:\-]\s*', '', text)
        
        # B. Extract Naturaleza (First word: INSTALACION, MEJORAMIENTO, etc.)
        naturaleza_match = re.search(r'^([A-ZÁÉÍÓÚÑ]+)', clean_name)
        naturaleza = naturaleza_match.group(1) if naturaleza_match else None
        
        # C. Extract Location (Sector/Neighborhood)
        # Usually starts after "EN EL/LA" or "DE LA" and ends before "DISTRITO"
        loc_match = re.search(r'(?:EN EL|EN LA|EN)\s+(.*?)(?=\s+DEL DISTRITO|\s+DISTRITO|\s+PROVINCIA|$)', clean_name, re.I)
        localidad = loc_match.group(1).strip() if loc_match else None
        
        # D. Extract District (DISTRITO DE ...)
        dist_match = re.search(r'DISTRITO DE\s+([\w\s]+?)(?=\s+PROVINCIA|\s+DEPARTAMENTO|\s+-|$)', clean_name, re.I)
        distrito = dist_match.group(1).strip() if dist_match else None
        
        # E. Extract Province (PROVINCIA DE ...)
        prov_match = re.search(r'PROVINCIA DE\s+([\w\s]+?)(?=\s+DEPARTAMENTO|\s+REGION|\s+-|$)', clean_name, re.I)
        provincia = prov_match.group(1).strip() if prov_match else None
        
        # F. Extract Departamento (Last part after the dash or keywords)
        # If there's a dash at the end, the last part is usually the Dept.
        parts = clean_name.split('-')
        departamento = parts[-1].strip() if len(parts) > 1 else None
        
        return pd.Series([cui, naturaleza, clean_name, localidad, distrito, provincia, departamento])

    # Apply extraction
    new_cols = ['cui', 'naturaleza', 'proyecto_nombre', 'localidad', 'distrito', 'provincia', 'departamento']
    df[new_cols] = df[source_col].apply(extract_details)
    
    # Optional: Filter out rows that didn't have a CUI (like total sum rows or headers)
    # df = df.dropna(subset=['cui'])
    
    return df

# Example Usage:
# df_cleaned = process_siaf_dataframe(your_raw_df)

In [51]:
df_2024=process_siaf_dataframe(df_2024)

In [52]:
df_2024.columns

Index(['Funcion', 'PIA', 'PIM', 'Certificación', 'Compromiso Anual',
       'Ejecución', 'Unnamed: 6', 'Unnamed: 7', 'Avance %', 'year', 'function',
       'source_file', 'Producto / Proyecto',
       'Actividad / Acción de Inversión / Obra', 'cui', 'naturaleza',
       'proyecto_nombre', 'localidad', 'distrito', 'provincia',
       'departamento'],
      dtype='object')

In [53]:
df_2024

,Funcion,PIA,PIM,Certificación,Compromiso Anual,Ejecución,Unnamed: 6,Unnamed: 7,Avance %,year,...,source_file,Producto / Proyecto,Actividad / Acción de Inversión / Obra,cui,naturaleza,proyecto_nombre,localidad,distrito,provincia,departamento
0,NaN,NaN,NaN,NaN,NaN,Atención de Compromiso Mensual,Devengado,Girado,NaN,2024,...,MPCT_FUNCION_2024.xlsx,NaN,NaN,None,None,None,None,None,None,None
1,"03: PLANEAMIENTO, GESTION Y RESERVA DE CONTING...",15458879.0,13876625.0,13743625.0,13584571.0,13578391,13535937,13513748,97.5,2024,...,MPCT_FUNCION_2024.xlsx,NaN,NaN,None,None,None,None,None,None,None
2,05: ORDEN PUBLICO Y SEGURIDAD,406092.0,528536.0,520263.0,519673.0,519673,519673,519673,98.3,2024,...,MPCT_FUNCION_2024.xlsx,NaN,NaN,None,None,None,None,None,None,None
3,07: TRABAJO,400000.0,1116240.0,1096783.0,1087356.0,1087356,1051121,1050841,94.2,2024,...,MPCT_FUNCION_2024.xlsx,NaN,NaN,None,None,None,None,None,None,None
4,08: COMERCIO,0.0,24500.0,24500.0,24500.0,22117,22117,22117,90.3,2024,...,MPCT_FUNCION_2024.xlsx,NaN,NaN,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,NaN,0.0,26071.0,26070.0,26070.0,26070,26070,25970,100.0,2024,...,MPCT_FUNCION_2024_viviendaDesarrolloUrbano.xlsx,2342967: CREACION DE LA PLAZA PRINCIPAL EN EL ...,NaN,2342967,CREACION,CREACION DE LA PLAZA PRINCIPAL EN EL CENTRO PO...,CENTRO POBLADO DE SAN JUAN DE LLAC-HUA,None,COTABAMBAS,APURIMAC
193,NaN,0.0,13002.0,13001.0,13001.0,13001,13001,13001,100.0,2024,...,MPCT_FUNCION_2024_viviendaDesarrolloUrbano.xlsx,2533719: MEJORAMIENTO DE LA PLAZA PRINCIAL DE ...,NaN,2533719,MEJORAMIENTO,MEJORAMIENTO DE LA PLAZA PRINCIAL DE CARMEN AL...,ALTO,CHALLHUAHUACHO,COTABAMBAS,DEPARTAMENTO DE APURIMAC
194,NaN,0.0,1000000.0,547645.0,542231.0,486651,486651,486651,48.7,2024,...,MPCT_FUNCION_2024_viviendaDesarrolloUrbano.xlsx,2548982: CREACION DE LA PLAZA PRINCIPAL DE LA ...,NaN,2548982,CREACION,CREACION DE LA PLAZA PRINCIPAL DE LA LOCALIDAD...,None,CHALLHUAHUACHO,COTABAMBAS,DEPARTAMENTO DE APURIMAC
195,NaN,0.0,128301.0,128301.0,128301.0,128301,128301,128136,100.0,2024,...,MPCT_FUNCION_2024_viviendaDesarrolloUrbano.xlsx,2617903: MEJORAMIENTO DEL SERVICIOS DE ESPACIO...,NaN,2617903,MEJORAMIENTO,MEJORAMIENTO DEL SERVICIOS DE ESPACIOS PUBLICO...,PUENTE UNION DEL BARRIO PAMPAÑA DE CENTRO POBL...,TAMBOBAMBA DE LA,COTABAMBAS DEL,None


In [54]:
from pathlib import Path
import pandas as pd

def load_spending_data_separated(base_dir, year):
    base_dir = Path(base_dir)
    year_folder = base_dir / f"SMPCT_{year}"
    
    if not year_folder.exists():
        raise FileNotFoundError(f"Folder not found: {year_folder}")

    # To store the separate outputs
    general_df = pd.DataFrame()
    categorized_dfs = []

    for file in year_folder.glob("*.xlsx"):
        filename = file.stem 
        parts = filename.split("_")

        # Logic to identify the "General" file vs the others
        # MPCT_FUNCION_2015.xlsx has exactly 3 parts
        if len(parts) == 3:
            try:
                print(f"Processing General file: {file.name}")
                general_df = load_file_with_dynamic_header(file)
                general_df["year"] = int(year)
                general_df["function"] = "general"
                general_df["source_file"] = file.name
            except Exception as e:
                print(f"Error reading general file {file.name}: {e}")
        
        else:
            # These are the files like _salud, _educacion, _sinProducto, etc.
            try:
                df = load_file_with_dynamic_header(file)
                df["year"] = int(year)
                df["function"] = "_".join(parts[3:]).lower()
                df["source_file"] = file.name
                categorized_dfs.append(df)
            except Exception as e:
                print(f"Error reading categorized file {file.name}: {e}")

    # Combine all category files into one, keeping general separate
    all_categories_df = pd.concat(categorized_dfs, ignore_index=True) if categorized_dfs else pd.DataFrame()

    return general_df, all_categories_df

# Usage
# df_general, df_others = load_spending_data_separated('your_path', 2015)

In [ ]:
df_2015 = load_spending_data_separated(r"D:\cg_inter_stuff\thesis_project\spending_data", 2015)


print(df_2015)

Processing General file: MPCT_FUNCION_2015.xlsx
Error reading categorized file ~$MPCT_FUNCION_2015_viviendaDesarrolloUrbano.xlsx: [Errno 13] Permission denied: 'D:\\cg_inter_stuff\\thesis_project\\spending_data\\SMPCT_2015\\~$MPCT_FUNCION_2015_viviendaDesarrolloUrbano.xlsx'


TypeError: 'module' object is not callable